In [2]:
import geopandas as gpd
import pandas as pd

## Open Europe Data

In [16]:
world_boundaries = gpd.read_file("data/world_boundaries.gpkg")

In [17]:
world_boundaries = world_boundaries[["geometry", "name", "iso3"]]

In [18]:
world_boundaries = world_boundaries.rename(
    columns={"name": "Area", "iso3": "Country Code"}
)

## Open pop data

In [23]:
country_pop_df = pd.read_csv(
    "data/API_SP.POP.TOTL_DS2_en_csv_v2_87/API_SP.POP.TOTL_DS2_en_csv_v2_87.csv",
    skiprows=4,
)

In [24]:
country_pop_df = country_pop_df[["Country Code", "2022"]]

In [27]:
world_boundaries_with_pop_gdf = world_boundaries.merge(
    country_pop_df, on="Country Code", how="left"
)

## Get FAO data

In [28]:
production_crops_df = pd.read_csv(
    "data/Production_Crops_Livestock_E_All_Data/Production_Crops_Livestock_E_All_Data.csv"
)

/tmp/ipykernel_76628/4000880986.py:1: DtypeWarning: Columns (11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59,62,65,68,71,74,77,80,83,86,89,92,95,98,101,104,107,110,113,116,119,122,125,128,131,134,137,140,143,146,149,152,155,158,161,164,167,170,173,176,179,182,185,188,191,194,197) have mixed types. Specify dtype option on import or set low_memory=False.
  production_crops_df = pd.read_csv(


In [29]:
sunflower_df = production_crops_df[production_crops_df["Item Code"] == 268][
    ["Area", "Y2022"]
]

## Merge Data

In [30]:
sunflower_gdf = world_boundaries.merge(sunflower_df, on="Area", how="right")

In [31]:
sunflower_gdf[sunflower_gdf["geometry"] == None]

,geometry,Area,Country Code,Y2022
12,None,Belgium-Luxembourg,NaN,NaN
13,None,Bolivia (Plurinational State of),NaN,90100.00
14,None,Bosnia and Herzegovina,NaN,32400.00
22,None,"China, mainland",NaN,304100.00
25,None,Czechia,NaN,96600.00
26,None,Czechoslovakia,NaN,NaN
60,None,Netherlands (Kingdom of the),NaN,207000.00
61,None,North Macedonia,NaN,3200.00
68,None,Republic of Moldova,NaN,229256.00
72,None,Serbia and Montenegro,NaN,NaN


In [65]:
to_replace = {
    "Turkey": "Türkiye",
    "Bosnia & Herzegovina": "Bosnia and Herzegovina",
    "Moldova, Republic of": "Republic of Moldova",
    "The former Yugoslav Republic of Macedonia": "North Macedonia",
    "Czech Republic": "Czechia",
    "Netherlands": "Netherlands (Kingdom of the)",
}
world_boundaries = world_boundaries.replace(to_replace)

In [66]:
sunflower_gdf = world_boundaries_with_pop_gdf.merge(sunflower_df, on="Area", how="left")

In [67]:
sunflower_gdf = sunflower_gdf[["geometry", "Y2022", "Area", "2022"]].fillna(0)
sunflower_gdf = sunflower_gdf.rename(columns={"Y2022": "OIL", "2022": "POPULATION"})
sunflower_gdf["OIL"] = sunflower_gdf["OIL"].astype(int)
sunflower_gdf["POPULATION"] = sunflower_gdf["POPULATION"].astype(int)

In [68]:
sunflower_gdf["OIL_PER_1000"] = (
    (sunflower_gdf["OIL"] / (sunflower_gdf["POPULATION"] / 1_000)).fillna(0)
).round(decimals=1)
sunflower_gdf["OIL_PER_100"] = (
    (sunflower_gdf["OIL"] / (sunflower_gdf["POPULATION"] / 100)).fillna(0)
).round(decimals=1)

In [69]:
sunflower_gdf.to_file("data/sunflower_oil.gpkg")